In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.11 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.


In [2]:
!pip uninstall -y bitsandbytes
!pip install bitsandbytes
!pip install --upgrade accelerate transformers trl


Found existing installation: bitsandbytes 0.40.2
Uninstalling bitsandbytes-0.40.2:
  Successfully uninstalled bitsandbytes-0.40.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 98.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
  Attempting uni

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# meta-llama/Llama-2-7b-chat-hf

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [6]:
import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

compute_dtype = torch.bfloat16  # Change based on your hardware (torch.float16 for lower-end GPUs)
use_4bit = True
bnb_4bit_quant_type = "nf4"
use_nested_quant = True
device_map = "auto"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Define the synthetic data generation function
def generate_example(prompt, temperature=0.4, max_new_tokens=200):
    input_text = f"Generate a customer support query and response related to: {prompt}"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # Generate output
    with torch.no_grad():
        output = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# List of query topics for generation
topics = [
    "Late delivery refund request",
    "Wrong item received",
    "Return policy inquiry",
    "Partial refund for damaged item",
    "Refund processing time",
    "Non-refundable item complaint",
    "Refund denied due to used product",
    "Gift return without receipt",
    "Refund for subscription cancellation",
    "Refund for out-of-stock item"
]

# Generate synthetic queries and responses
for i, topic in enumerate(topics, 1):
    print(f"\n**Example {i}: {topic}**")
    example = generate_example(topic)
    print(example)


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]


**Example 1: Late delivery refund request**
Generate a customer support query and response related to: Late delivery refund request
 everybody knows that the delivery time is usually 3-5 business days, but in some cases, it may take longer. However, if your order has not arrived within the estimated delivery time frame, you may be eligible for a refund. Please provide us with your order number and a detailed explanation of why you are requesting a refund. We will review your request and get back to you as soon as possible.

Query: "I ordered a product on Monday and it still hasn't arrived. Can I get a refund?"

Response: "Thank you for reaching out to us regarding the late delivery of your order. We apologize for any inconvenience this has caused. Unfortunately, our estimated delivery time frame was not met in this case, and we understand your frustration. Please provide us with your order number so we can review your request for a refund. We will get back to you as soon as possible w

In [33]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Load LLaMA 2 model (ensure you have access to Meta's model on Hugging Face)
# MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"  # Change based on your model
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

# Function to generate a synthetic query-response pair
def generate_example(topic, temperature=0.4, max_new_tokens=200):
    input_text = f"Generate a customer query and support response related to: {topic}"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # Generate output
    with torch.no_grad():
        output = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9
        )

    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Extract query and response from the output
    if "Customer:" in response_text and "Support:" in response_text:
        try:
            query = response_text.split("Customer:")[1].split("Support:")[0].strip()
            response = response_text.split("Support:")[1].strip()
        except IndexError:
            query, response = response_text, "Error extracting response"
    else:
        query, response = response_text, "Could not extract structured response."

    return {"query": query, "response": response}

# List of query topics
topics = [
    "Late delivery refund request",
    "Wrong item received",
    "Return policy inquiry",
    "Partial refund for damaged item",
    "Refund processing time",
    "Non-refundable item complaint",
    "Refund denied due to used product",
    "Gift return without receipt",
    "Refund for subscription cancellation",
    "Refund for out-of-stock item"
]

# Dictionary to store generated responses
synthetic_data = {}

# Generate 10 queries and responses per topic
for topic in topics:
    print(f"\nGenerating examples for: {topic}")
    synthetic_data[topic] = [generate_example(topic) for _ in range(2)]

# Print the structured data
for topic, examples in synthetic_data.items():
    print(f"\n**Topic: {topic}**")
    for i, ex in enumerate(examples, 1):
        print(f"{i}. Query: {ex['query']}")
        print(f"   Response: {ex['response']}\n")



Generating examples for: Late delivery refund request

Generating examples for: Wrong item received

Generating examples for: Return policy inquiry

Generating examples for: Partial refund for damaged item

Generating examples for: Refund processing time

Generating examples for: Non-refundable item complaint

Generating examples for: Refund denied due to used product

Generating examples for: Gift return without receipt

Generating examples for: Refund for subscription cancellation

Generating examples for: Refund for out-of-stock item

**Topic: Late delivery refund request**
1. Query: Generate a customer query and support response related to: Late delivery refund request
 hopefully this message finds you well. I am writing to request a refund for my recent purchase that was supposed to be delivered on the 15th of this month but unfortunately, it didn't arrive on time. I have checked my tracking information and it shows that the package is still in transit, but it's now the 20th and 

In [34]:
synthetic_data

{'Late delivery refund request': [{'query': "Generate a customer query and support response related to: Late delivery refund request\n hopefully this message finds you well. I am writing to request a refund for my recent purchase that was supposed to be delivered on the 15th of this month but unfortunately, it didn't arrive on time. I have checked my tracking information and it shows that the package is still in transit, but it's now the 20th and I have not received any updates on its delivery status. I have also tried contacting the seller directly but have not received any response. I would greatly appreciate it if you could look into this matter and provide me with a refund as soon as possible. I understand that delays can happen, but as a customer, I expect timely delivery and communication from the seller. Please let me know if there's anything else I need to do on my end to resolve this issue. Thank you for your time and attention. Sincerely, [Customer Name] Dear [Customer Name],

In [30]:
import json
merged_data = []
for topic, entries in synthetic_data.items():
    merged_data.extend(entries)

# Save merged data to a JSON file
output_filename = "merged_synthetic_data2.json"
# with open(output_filename, "w", encoding="utf-8") as f:
#     json.dump(merged_data, f, indent=4, ensure_ascii=False)

# print(f"\n✅ Merged data saved to {output_filename}")
query_list = [entry["query"] for entry in merged_data]
(query_list)

['Generate a customer query and support response related to: Late delivery refund request. Begriffe und Definitionen. In this case, the customer is requesting a refund for a late delivery of their order.\n\nCustomer Query:\n\nDear [Company Name],\n\nI am writing to request a refund for my recent order that was supposed to be delivered on [date of delivery]. However, I have not received my order yet and it is now [current date]. I understand that delays can happen, but I am getting concerned as I need the items urgently. Could you please look into this matter and provide a refund as soon as possible?\n\nThank you for your attention to this matter.\n\nSincerely,\n[Customer Name]\n\nSupport Response:\n\nDear [Customer Name],\n\nThank you for reaching out to us regarding the late delivery of your order. We apologize for any inconvenience this has caused and are working to resolve the issue as soon as possible.\n\nWe',
 'Generate a customer query and support response related to: Late delive

In [35]:
import re

def preprocess_queries(query_list):
    structured_data = []
    for text in query_list:
        # Extracting the customer query (between "Customer Query:" and "Support Response:")
        query_match = re.search(r"Customer Query:\s*(.*?)\s*Support Response:", text, re.DOTALL)
        query = query_match.group(1).strip() if query_match else "Query not found"

        # Extracting the support response (everything after "Support Response:")
        response_match = re.search(r"Support Response:\s*(.*)", text, re.DOTALL)
        response = response_match.group(1).strip() if response_match else "Response not found"

        structured_data.append({"query": query, "response": response})
    
    return structured_data

# Process the data
structured_queries = preprocess_queries(query_list)

# Print the result
print(structured_queries)

[{'query': 'Dear [Company Name],\n\nI am writing to request a refund for my recent order that was supposed to be delivered on [date of delivery]. However, I have not received my order yet and it is now [current date]. I understand that delays can happen, but I am getting concerned as I need the items urgently. Could you please look into this matter and provide a refund as soon as possible?\n\nThank you for your attention to this matter.\n\nSincerely,\n[Customer Name]', 'response': 'Dear [Customer Name],\n\nThank you for reaching out to us regarding the late delivery of your order. We apologize for any inconvenience this has caused and are working to resolve the issue as soon as possible.\n\nWe'}, {'query': '"I ordered a package on [date] but it still hasn\'t arrived. Can I get a refund? I need the item urgently and can\'t afford to wait any longer. Please let me know what steps I can take to get my refund as soon as possible."', 'response': 'Dear [Customer Name],\n\nThank you for reach

In [36]:

output_filename = "merged_synthetic_data4.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(structured_queries, f, indent=4, ensure_ascii=False)

print(f"\n✅ Merged data saved to {output_filename}")


✅ Merged data saved to merged_synthetic_data4.json


In [ ]:
import os
import json

# Function to remove duplicates based on specific keys
def remove_duplicates(data, keys):
    seen = {}
    for item in data:
        identifier = tuple(item[key] for key in keys)
        if identifier not in seen or len(item['response']) > len(seen[identifier]['response']):
            seen[identifier] = item
    return list(seen.values())

# Load data from the first JSON file
with open('json/merged_synthetic_data3.json', 'r', encoding='utf-8') as file:
    data1 = json.load(file)

# Load data from the second JSON file
with open('json/merged_synthetic_data4.json', 'r', encoding='utf-8') as file:
    data2 = json.load(file)
    
with open('merged_filtered.json', 'r', encoding='utf-8') as file:
    data3 = json.load(file)

# Merge the data
merged_data = data1 + data2 + data3

# Remove entries where 'query' is 'Query not found'
filtered_data = [item for item in merged_data if item['query'] != 'Query not found']

# Remove duplicates based on 'query'
unique_data = remove_duplicates(filtered_data, ['query'])

# Save the merged and filtered data to a new JSON file
with open('merged_filtered_data.json', 'w', encoding='utf-8') as outfile:
    json.dump(unique_data, outfile, indent=4)

print("Merged and filtered JSON files, and removed duplicates.")